In [1]:
import os
from tools import us_plots
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.platypus import Table, Paragraph, Spacer, TableStyle, BaseDocTemplate, Frame, PageTemplate
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus.flowables import Image, Spacer, PageBreak
from reportlab.platypus.paragraph import Paragraph
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

In [2]:
COMPANY_NAME = "LOREM IPSUM<br />COMPANY" # Use <br /> to break lines
AUTHOR = "HGNX"
AUTHOR_EMAIL = "contact@hgk.im"
DATE = "Jan 1, 2023"
FILE_NAME = "output.pdf"

In [3]:
class MyDocTemplate(BaseDocTemplate):
    def __init__(self, filename, **kwargs):
        BaseDocTemplate.__init__(self, filename, **kwargs)
        frame_x = (A4[0] - 6*inch) / 2
        template = PageTemplate('normal', [Frame(frame_x, 1*inch, 6*inch, 10*inch)])
        self.addPageTemplates(template)

    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__ == Paragraph:
            text = flowable.getPlainText()
            self.notify('TOCEntry', (text, self.page))

In [4]:
assets_folder = os.path.join(os.getcwd(), "assets")
font = os.path.join(assets_folder, "NotoSansKR[wght].ttf")
pdfmetrics.registerFont(TTFont('NotoSansKR', font))

doc = MyDocTemplate(FILE_NAME, pagesize=A4)

In [5]:
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Center', alignment=1, fontSize=14, spaceAfter=20, textColor=colors.black, fontWeight='bold'))

left_style = ParagraphStyle(name='Left', parent=styles['Normal'], alignment=0)
center_style = ParagraphStyle(name='Center', parent=styles['Normal'], alignment=1, fontSize=14, spaceAfter=20, textColor=colors.black, fontWeight='bold', leading=18)
right_style = ParagraphStyle(name='Right', parent=styles['Normal'], alignment=2, fontSize=8, leading=10)

header = [
    [
        Paragraph(COMPANY_NAME, left_style),
        "", 
        Paragraph("<b>STOCK MARKET<br />DAILY REPORT</b>", center_style),
        "", 
        Paragraph(f"{AUTHOR}<br />{AUTHOR_EMAIL}<br />{DATE}", right_style)
    ]
]

header_table_style = TableStyle([
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 15),
    ('LINEBELOW', (0, 0), (-1, 0), 1, colors.grey),
])

header_table = Table(header, colWidths=[1.5*inch, 0.5*inch, 3*inch, 0.5*inch, 1.5*inch])
header_table.setStyle(header_table_style)

story = [header_table, Spacer(1, 0.3*inch)]

In [6]:
caption_style = ParagraphStyle(name='Caption', parent=styles['Normal'], fontSize=7, alignment=0)
caption_indices = Paragraph("● Global Stock Market", caption_style)
caption_ficc = Paragraph("● FICC", caption_style)
caption_gainers = Paragraph("● Top Gainers", caption_style)
caption_losers = Paragraph("● Top Losers", caption_style)

In [7]:
table_indices_data = [
    ["Country", "Closed(pt)", "1D(%)", "1W(%)", "MTD(%)", "1M(%)", "YTD(%)"],
    ["DOW", "34,509.03", "0.33", "2.29", "1.13", "1.56", "4.11"],
    ["S&P500", "4,505.42", "-0.10", "2.42", "2.48", "3.04", "17.34"],
    ["NASDAQ", "14,113.70", "-0.18", "3.32", "3.84", "3.58", "34.85"],
    ["RUSSELL 2000", "1,931.09", "-1.01", "3.56", "2.63", "3.04", "9.64"],
    ["STOXX 600", "460.83", "-0.11", "2.94", "0.92", "-0.88", "8.46"],
    ["영국(FTSE)", "7,434.57", "-0.08", "2.45", "-0.50", "-2.21", "-0.23"],
    ["프랑스(CAC)", "7,374.54", "0.06", "3.69", "0.85", "0.63", "13.91"],
    ["독일(DAX)", "16,105.07", "-0.22", "3.22", "0.99", "-1.26", "15.67"],
    ["상해종합", "3,237.70", "0.04", "1.29", "1.74", "0.27", "4.81"],
    ["홍콩H", "19,413.78", "0.33", "5.71", "2.53", "0.03", "-1.86"],
    ["항셍지수", "6,558.88", "0.23", "5.81", "2.06", "-0.27", "-2.18"],
    ["니케이225", "32,391.26", "-0.09", "0.01", "-2.54", "-3.32", "24.13"],
    ["KOSPI", "2,628.30", "1.43", "4.02", "3.07", "0.35", "17.52"]
]

table_ficc_data = [
    ["Category", "Closed(pt)", "1D(bp, %)", "1W(bp, %)", "MTD(bp, %)", "1M(bp, %)", "YTD(bp, %)"],
    ["미국채2년(%)", "4.77", "13.5bp", "-18.0bp", "-9.4bp", "7.8bp", "34.0bp"],
    ["미국채10년(%)", "3.83", "6.9bp", "-22.9bp", "-0.6bp", "4.6bp", "-4.3bp"],
    ["국고채3년(%)", "3.62", "1.3bp", "-12.0bp", "2.0bp", "7.6bp", "-11.0bp"],
    ["독일10년(%)", "2.51", "3.2bp", "-12.5bp", "9.5bp", "6.1bp", "-5.6bp"],
    ["일본10년(%)", "0.47", "0.7bp", "5.3bp", "9.1bp", "4.9bp", "6.1bp"],
    ["달러인덱스(pt)", "99.91", "0.14", "-2.31", "-3.32", "-2.95", "-3.49"],
    ["NDF 환율(1M)", "1,267.43", "0.19", "-2.32", "-4.00", "-0.37", "0.49"],
    ["유로/달러", "1.12", "0.02", "2.38", "3.34", "3.67", "4.89"],
    ["위안/달러", "7.14", "-0.10", "-1.15", "-1.46", "-0.28", "3.53"],
    ["엔/달러", "138.80", "0.54", "-2.40", "-4.12", "-0.92", "5.86"],
    ["WTI(NYMEX)", "75.42", "-1.91", "2.11", "7.96", "10.17", "-4.96"],
    ["구리(LME)", "8,663.25", "-0.31", "3.51", "5.88", "1.73", "3.48"],
    ["금(NYMEX)", "1,964.40", "0.03", "1.65", "2.42", "-0.23", "4.85"]
]

In [8]:
table_gainers_data = [
    ["Ticker", "Company", "Price", "Change", "Change(%)"],
    ["PKG", "Packaging Corp of America", "152.65", "13.98", "10.08"],
    ["MSCI", "MSCI Inc", "548.20", "45.45", "9.04"],
    ["GE", "General Electric Co", "117.16", "6.91", "6.27"],
    ["PHM", "Pultegroup Inc", "83.42", "4.89", "6.23"],
    ["MMM", "3M Co", "109.83", "5.56", "5.33"],
    ["IP", "International Paper Co", "33.93", "1.56", "4.82"],
    ["WRK", "Westrock Co", "32.06", "1.30", "4.23"],
    ["FCX", "Freeport-McMoRan Inc", "43.69", "1.59", "3.78"],
    ["NUE", "Nucor Corp", "172.88", "6.23", "3.74"],
    ["ADM", "Archer-Daniels-Midland Co", "86.05", "2.94", "3.54"]
]

table_losers_data = [
    ["Ticker", "Company", "Price", "Change", "Change(%)"],
    ["RTX", "Rtx Corp", "87.10", "-9.91", "-10.22"],
    ["ALK", "Alaska Air Group Inc", "48.18", "-5.15", "-9.66"],
    ["IVZ", "Invesco Ltd", "16.62", "-1.34", "-7.46"],
    ["LW", "Lamb Weston Holdings Inc", "105.00", "-7.69", "-6.82"],
    ["LUV", "Southwest Airlines Co", "35.53", "-1.67", "-4.49"],
    ["DOV", "Dover Corp", "146.51", "-5.58", "-3.67"],
    ["MTD", "Mettler-Toledo International Inc", "1,312.92", "-49.10", "-3.60"],
    ["USB", "US Bancorp", "37.86", "-1.40", "-3.57"],
    ["GM", "General Motors Co", "37.92", "-1.38", "-3.51"],
    ["VFC", "VF Corp", "18.89", "-0.63", "-3.23"]
]

In [9]:
table_style_overview = [
    ('LINEABOVE', (0, 1), (-1, 1), 0.5, colors.grey),
    ('BOX', (0, 0), (-1, -1), 0, colors.transparent),
    ('GRID', (0, 0), (-1, -1), 0, colors.transparent),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, 0), 'RIGHT'),
    ('ALIGN', (0, 0), (0, -1), 'LEFT'),
    ('ALIGN', (1, 1), (-1, -1), 'RIGHT'),
    ('FONTNAME', (0, 0), (-1, -1), 'NotoSansKR'),
    ('FONTSIZE', (0, 0), (-1, -1), 5),
    ('LEFTPADDING', (0, 0), (-1, -1), 1),
    ('RIGHTPADDING', (0, 0), (-1, -1), 1),
]

table_style_movers = [
    ('LINEABOVE', (0, 1), (-1, 1), 0.5, colors.grey),
    ('BOX', (0, 0), (-1, -1), 0, colors.transparent),
    ('GRID', (0, 0), (-1, -1), 0, colors.transparent),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('ALIGN', (0, 0), (-1, 0), 'RIGHT'),
    ('ALIGN', (0, 0), (1, -1), 'LEFT'),
    ('ALIGN', (2, 1), (-1, -1), 'RIGHT'),
    ('FONTNAME', (0, 0), (-1, -1), 'NotoSansKR'),
    ('FONTSIZE', (0, 0), (-1, -1), 5),
    ('LEFTPADDING', (0, 0), (-1, -1), 1),
    ('RIGHTPADDING', (0, 0), (-1, -1), 1),
]

In [10]:
additional_space = 0.3*inch

total_table_width = 6*inch - 0.1*inch - additional_space
single_table_width = total_table_width / 2
column_width = single_table_width / 6
table_row_height = 0.15*inch

In [11]:
table_overview_column_width = single_table_width / 6
table_indices = Table(table_indices_data, [column_width for _ in range(7)], [table_row_height for _ in range(len(table_indices_data))])
table_indices.setStyle(table_style_overview)

table_ficc = Table(table_ficc_data, [column_width for _ in range(7)], [table_row_height for _ in range(len(table_ficc_data))])
table_ficc.setStyle(table_style_overview)

table_overview_data = [[caption_indices, Spacer(0.1*inch, 0), caption_ficc],
                       [table_indices, Spacer(0.1*inch, 0), table_ficc]]
table_overview = Table(table_overview_data)

In [12]:
mover_first_column_width = column_width * 0.9
mover_second_column_width = column_width + (column_width * 1.5)
mover_other_columns_width = column_width * 1.2

table_mover_width = [mover_first_column_width, mover_second_column_width] + [mover_other_columns_width for _ in range(3)]
table_gainers = Table(table_gainers_data, table_mover_width, [table_row_height for _ in range(len(table_gainers_data))])
table_gainers.setStyle(table_style_movers)

table_losers = Table(table_losers_data, table_mover_width, [table_row_height for _ in range(len(table_losers_data))])
table_losers.setStyle(table_style_movers)

table_mover_data = [[caption_gainers, Spacer(0.1*inch, 0), caption_losers],
                    [table_gainers, Spacer(0.1*inch, 0), table_losers]]
table_mover = Table(table_mover_data)

In [13]:
us_plots.create_plots()

nasdaq_graph = Image(os.path.join(assets_folder, "nasdaq.png"), 2.4*inch, 1.6*inch)
sp500_graph = Image(os.path.join(assets_folder, "sp500.png"), 2.4*inch, 1.6*inch)
sectors_graph = Image(os.path.join(assets_folder, "sectors.png"), 2.4*inch, 1.6*inch)

graph_data = [
    nasdaq_graph, 
    Spacer(0.1*inch, 0), 
    sp500_graph, 
    Spacer(0.1*inch, 0), 
    sectors_graph
]
graph_table = Table([graph_data], colWidths=[2.4*inch, 0.001*inch, 2.4*inch, 0.001*inch, 2.4*inch])

In [14]:
story.append(table_overview)
story.append(Spacer(1, 0.05*inch))
story.append(graph_table)
story.append(Spacer(1, 0.05*inch))
story.append(table_mover)
doc.build(story)

In [15]:
image_files = ["nasdaq.png", "sp500.png", "sectors.png"]
for image_file in image_files:
    image_path = os.path.join(assets_folder, image_file)
    if os.path.exists(image_path):
        os.remove(image_path)